# MGMTMSA 408 -- Lecture 4: Assortment Optimization

In this notebook, we will be exploring the topic of assortment/product line optimization using a data set from Timbuk2.

## Setting up the data
First, let's extract the customer information. 

In [1]:
# First, let's import some packages.
import numpy as np
import pandas as pd

# Read the partworth matrix:
partworths = pd.read_csv("partworths_small_v2.csv",  header=None)

# Convert to a numpy array to make life easier: 
partworths = np.asarray(partworths)
nCustomers = partworths.shape[0]
print("Number of customers: ", nCustomers)
print("Shape: ", partworths.shape)

Number of customers:  330
Shape:  (330, 10)


The partworth array contains information about the partworths (attribute level utilities) for the 330 customers in our sample. The rows correspond to different customers, while the columns correspond to different attributes. The attributes in this problem are as follows:

|Attribute Num.|Attribute|
|-|-|
|1|Price|
|2|Size (0 = normal, 1 = large)|
|3|Color (0 = black, 1 = red)|
|4|Logo (0 = no logo, 1 = logo)|
|5|Handle (0 = no handle, 1 = has handle)|
|6|PDA holder (0 = no holder, 1 = has holder)|
|7|Cellphone holder (0 = no holder, 1 = has holder)|
|8|Mesh pocket (0 = no pocket, 1 = has pocket)|
|9|Velcro flap (0 = no flap, 1 = has flap)|
|10|Protective boot (0 = no boot, 1 = has boot)|


The columns in partworths indicate the following about the customer utilities for these attributes:

|Column|Meaning
| :--- | :--- |
|1|Difference in utility of a price of \\$ $x$ + 1 to a price of \\$ $x$|
|2|" " " of large bag relative to normal size bag|
|3|" " " of red bag relative to black bag|
|4|" " " of bag with logo relative to bag with no logo|
|5|" " " of bag with handle relative to bag with no handle|
|6|" " " of bag with PDA holder relative to bag with no PDA holder|
|7|" " " of bag with cellphone holder relative to bag with no cellphone holder|
|8|" " " of bag with mesh pocket relative to bag with no pocket|
|9|" " " of bag with velcro flap relative to bag with no flap|
|10|" " " of bag with protective boot relative to bag with no boot|

Some examples:

In [17]:
# Utility of customer 10 for attribute 6 (PDA holder) and attribute 9 (velcro flap):
print("Customer 10: ")
print(partworths[9,5])
print(partworths[9,8])

# Utility of customer 20 for attribute 6 (PDA holder):
print("Customer 20: ")
print(partworths[19,5])
print(partworths[19,8])

# The above hopefully gives you a taste of why this problem is going to be difficult!

# Customer's sensitivity to price: (always negative)
print(partworths[9,0])
print(partworths[19,0])


print( partworths[9,] / np.abs(partworths[9,0]))

Customer 10: 
11.79
-4.45
Customer 20: 
-17.41
46.25
-1.67833333333333
-0.327333333333333
[-1.         21.1876862  15.29493545 40.4448858  24.45283019  7.02482622
 11.88083416 23.21350546 -2.65143992 34.83813307]


Now that we have the partworth information, let's load the candidate products that we are going to consider. The columns correspond to products, while the rows indicate the attributes. In the set of products that we will consider, the price is discretized between \\$70 and \\$100 in \\$5 increments (i.e., the price can be \\$70, \\$75, ..., \\$100). With this discretization, the file grand_product_matrix_small_v2.csv contains only a subset of 100 products. (Note that the total number of attribute combinations, with this discretization of price, is 3584; using all 3584 products renders the problem extremely difficult, so we will restrict ourselves to only 100 product.)

In [3]:
# Load the product matrix
grand_product_matrix = pd.read_csv("grand_product_matrix_small_v2.csv", header = None)
grand_product_matrix = np.asarray(grand_product_matrix)

# Get the number of products
nProducts = grand_product_matrix.shape[1]

print("Shape: ", grand_product_matrix.shape)

# Check product 1 
print("Product 1 attributes: ")
print(grand_product_matrix[:,0])
# Bag priced at $70, normal size black bag with a velcro flap

# Check product 20
print("Product 20 attributes: ")
print(grand_product_matrix[:,19])
# Bag priced at $100, normal size black bag, with logo, handle and protective boot

Shape:  (10, 100)
Product 1 attributes: 
[70  0  0  0  0  0  0  0  1  0]
Product 20 attributes: 
[100   0   0   1   1   0   0   0   0   1]


To make our lives easier, let's define a simple function that can take an attribute array and output a nice English description of the product:

In [18]:
# print_attributes function to return string with product attributes.
def print_attributes(attr_vec):
    attr_string = "Bag: ";
    attr_string += "$" + str(attr_vec[0])
    attr_string += (", Large" if attr_vec[1] == 1 else ", Normal")
    attr_string += (", Grey/Red" if attr_vec[2] == 1 else ", Black")
    attr_string += (", Logo" if attr_vec[3] == 1 else "")
    attr_string += (", Handle" if attr_vec[4] == 1 else "")
    attr_string += (", PDA Holder" if attr_vec[5] == 1 else "")
    attr_string += (", Cellphone Holder" if attr_vec[6] == 1 else "")
    attr_string += (", Mesh Pocket" if attr_vec[7] == 1 else "")
    attr_string += (", Velcro Flap" if attr_vec[8] == 1 else "")
    attr_string += (", Boot" if attr_vec[9] == 1 else "")
    return attr_string

# Test it out:
print("Product 1 attributes: ")
print( print_attributes(grand_product_matrix[:,0]) )

print("Product 20 attributes: ")
print( print_attributes(grand_product_matrix[:,19]) )

Product 1 attributes: 
Bag: $70, Normal, Black, Velcro Flap
Product 20 attributes: 
Bag: $100, Normal, Black, Logo, Handle, Boot


We still have a couple of things that we need to do before we can move on to testing out sets of products. We need to obtain the marginal profit of each product and we need to obtain the utilities of the products.

The following is known about the marginal incremental cost of each attribute:

|Attribute Num.|Attribute|Marginal incremental cost|
|-|-|-|
|2|Size (0 = normal, 1 = large)| \$3.50|
|3|Color (0 = black, 1 = red)| \$0.00|
|4|Logo (0 = no logo, 1 = logo)| \$2.00|
|5|Handle (0 = no handle, 1 = has handle)| \$3.50 |
|6|PDA holder (0 = no holder, 1 = has holder)| \$3.00|
|7|Cellphone holder (0 = no holder, 1 = has holder)| \$3.00|
|8|Mesh pocket (0 = no pocket, 1 = has pocket)| \$2.00|
|9|Velcro flap (0 = no flap, 1 = has flap)| \$3.50|
|10|Protective boot (0 = no boot, 1 = has boot)| \$4.50|

Let's now calculate the marginal profit of each of the 100 candidate products.

In [19]:
# Incremental costs of the non-price attributes:
incremental_cost = np.array([3.5, 0, 2, 3.5, 3, 3, 2, 3.5, 4.5])

# Calculate the marginal profit
marginal_profit = grand_product_matrix[0,:] - np.dot(incremental_cost, grand_product_matrix[1:,:] )
print(marginal_profit)

# Let's take an example just to make sure this worked correctly:
print(grand_product_matrix[:,99])
print(marginal_profit[99])
print(75 - 3.5 - 3.5 - 3 - 3.5 - 4.5)

[66.5 62.  60.5 53.5 61.5 48.5 61.5 55.5 59.  58.5 59.5 61.  53.5 53.5
 63.5 60.  96.5 90.5 88.5 90.  89.5 94.  89.  83.5 88.  80.5 88.  84.5
 91.  88.  88.  88.5 84.  83.  94.5 85.5 85.  86.5 84.5 84.5 90.  80.5
 86.5 87.  92.  75.5 85.5 86.  80.  73.  89.5 73.5 70.  85.5 81.5 84.5
 82.5 77.  79.  83.5 79.  79.5 77.  67.  72.  80.  76.5 65.  73.  63.5
 64.  66.5 72.5 64.  65.5 73.5 70.  71.5 78.  68.5 57.  66.5 62.5 71.
 68.  60.5 68.5 65.  62.  75.  68.5 67.5 66.5 72.  63.  62.  58.5 61.5
 61.5 57. ]
[75  1  0  0  1  1  0  0  1  1]
57.0
57.0


Great! We have the marginal profits. Now let's get the utility of each product. We need a matrix where the rows correspond to customers, and the columns correspond to products. Numpy can again help us with this.

In [6]:
# Calculate the utilities matrix:
utilities_mat = np.dot(partworths, grand_product_matrix)
# Inspect the dimensions:
print("Dimensions: ",utilities_mat.shape)

# Again, let's take an example to make sure things worked ok:
print(grand_product_matrix[:,99])
print(partworths[49,])
print(75 * (-0.005) + 1*(50.09) + 0 *(-99.85) + 0 * (-86.59) + 1 * 71.4 + 1 * (-24.1) + 0*(-0.11) + 0*(0.28) + 1 * (-24.27) + 1 * 0.34)
print(utilities_mat[49,99])

Dimensions:  (330, 100)
[75  1  0  0  1  1  0  0  1  1]
[-5.000e-03  5.009e+01 -9.985e+01 -8.659e+01  7.140e+01 -2.410e+01
 -1.100e-01  2.800e-01 -2.427e+01  3.400e-01]
73.08500000000002
73.08500000000002


So we now have our utilities!

We need one final thing before we move on to computing some expected profits. Our model of customer choice assumes that customers can always choose to not purchase any of the products. The data we have does not explicitly have a utility for this possibility. 

A common approach to dealing with this is to assume that the customers can choose certain competitive offerings. As mentioned in the slides, we will assume that we have 3 competitive offerings: a low-end bag, a mid-range bag and a high-end bag. The no-purchase utility for a given customer will thus correspond to the highest utility of these three.

In [7]:
# Create the attributes for the no-purchase options:
competitive_products = np.array([[70, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                                 [85, 0, 0, 0, 0, 1, 1, 1, 1, 1],
                                 [100, 1, 1, 1, 1, 1, 1, 1 ,1, 1]])
# Take the transpose to make it compatible with our partworths matrix:
competitive_products = competitive_products.transpose()
print(competitive_products)

# Compute the utility of each of these competitive products for each customer:
competitive_utilities = np.dot(partworths, competitive_products)
print(competitive_utilities)

# To get the no-purchase utilities, take the max along the product dimension:
nopurchase_utilities = competitive_utilities.max(axis = 1)

# Finally, let us tack these no-purchase utilities to the main utility matrix:
utilities_mat = np.concatenate( (utilities_mat, nopurchase_utilities[:,None]) , axis = 1)

# Check that this worked:
print(utilities_mat.shape)
print(utilities_mat[0,100])
print(nopurchase_utilities[0])

[[ 70  85 100]
 [  0   0   1]
 [  0   0   1]
 [  0   0   1]
 [  0   0   1]
 [  0   1   1]
 [  0   1   1]
 [  0   1   1]
 [  0   1   1]
 [  0   1   1]]
[[-1.57406667e+02 -1.17536667e+02 -2.84296667e+02]
 [-2.32096667e+02 -4.77601667e+02 -6.88626667e+02]
 [-1.77846667e+02 -7.31666667e+01  4.62733333e+01]
 [-1.21566667e+01  1.03138333e+02  2.03593333e+02]
 [-1.08243333e+02  1.60816667e+01  1.45936667e+02]
 [-2.31490000e+02 -1.45445000e+02 -1.15370000e+02]
 [-5.49500000e+01 -1.01850000e+01  1.22840000e+02]
 [-5.42266667e+01  3.12933333e+01  1.08743333e+02]
 [-2.13290000e+02 -1.40975000e+02 -2.94300000e+01]
 [-1.17483333e+02 -1.79483333e+01  1.27026667e+02]
 [-1.25300000e+02  1.67070000e+02  1.21930000e+02]
 [-4.26300000e+01  4.19850000e+01  1.06040000e+02]
 [-2.07200000e+02 -4.95860000e+02 -4.19930000e+02]
 [-9.06266667e+01 -4.18166667e+01 -5.82766667e+01]
 [-2.15996667e+02 -2.01991667e+02 -1.56616667e+02]
 [-2.82333333e+01 -9.58533333e+01 -1.88633333e+01]
 [-1.32136667e+02  1.10883333e+01

## Computing first-choice expected profits and choice probabilities

Now that we have figured out the marginal profits and utilities of all of the products, let's now move on to calculating the expected profit of different product lines, as well as the choice probabilities.

Our first function below will compute the expected profit of a given product line (an array of products).

In [8]:
# Function to compute expected profit
# S - the product line, represented as an array of integers
# Precondition: S is an array of integers containing only numbers between 0 and 99, with no repetitions.
def expected_profit(S):
    # Add the no-purchase option:
    # NB: the products are numbered from 0 to 99. Index 100 ( = nProducts) will correspond to the no-purchase option. 
    S2 = np.append(S, nProducts)
    
    choice_probabilities = {}
    for i in S2:
        choice_probabilities[i] = 0.0;
    
    for k in range(nCustomers):
        ind = np.argmax( utilities_mat[k,S2])
        choice_probabilities[ S2[ind] ] += 1.0/nCustomers
        
    exp_profit = sum( [choice_probabilities[i] * marginal_profit[i] for i in S])
        
    return exp_profit, choice_probabilities


# Let's see one simple example -- we only offer product 1.
exp_profit, choice_probabilities = expected_profit( [0] )
print("Expected profit: ", exp_profit)
print("Choice probabilities: ", choice_probabilities)

# Another example: we offer 20, 30, 52:
exp_profit, choice_probabilities = expected_profit( [19, 29, 51] )
print("Expected profit: ", exp_profit)
print("Choice probabilities: ", choice_probabilities)

Expected profit:  9.874242424242427
Choice probabilities:  {0: 0.14848484848484853, 100: 0.8515151515151551}
Expected profit:  32.289393939393946
Choice probabilities:  {19: 0.15454545454545457, 29: 0.08484848484848487, 51: 0.14848484848484853, 100: 0.6121212121212142}


## Testing common-sense product lines

Let's now try to see how different product lines will do in terms of profit.

Our first product line will be to offer everything. Let's see what happens when we do this:

In [9]:
S_all = np.array([i for i in range(nProducts)])
exp_profit_all, choice_probabilities_all = expected_profit( S_all )
print("Expected profit (offer everything): ", exp_profit_all)

Expected profit (offer everything):  59.72878787878786


If we offer all the 100 products, we will get an expected per-customer profit of \\$59.73. 

Let's see a different product line. Suppose that for every customer, we could offer their most preferred product. How much would we get with this product line?

In [10]:
# Use numpy's function argmax to obtain the element with the highest value.
# Note: 0:nProducts will leave out column nProducts (= 100, which is where we are storing
# the no-purchase option.)
S_mostpref = np.argmax(utilities_mat[:,0:nProducts], axis = 1)

S_mostpref = np.unique(S_mostpref)
print(S_mostpref)

exp_profit_mostpref, choice_probabilities_mostpref = expected_profit( S_mostpref )
print("Expected profit (offer most preferred): ", exp_profit_mostpref)

[ 0  1  2  3  4  5  6  7  9 10 11 12 13 18 19 20 22 25 33 39 41 49 51 52
 58 59 62 63 64 66 67 69 70 71 73 74 76 79 80 81 82 84 85 87 88 89 90 92
 94 95 96 97 98 99]
Expected profit (offer most preferred):  59.72878787878786


**Question:** Why did this happen?

Let's see some more. A reasonable suggestion might be to restrict ourselves to all products with a price of \\$100.

In [11]:
# Get all products priced at $100:
S_price100 = [i for i in range(100) if grand_product_matrix[0,i] == 100 ]

exp_profit_price100, choice_probabilities_price100 = expected_profit( S_price100 )
print("Expected profit (offer all priced at $100): ", exp_profit_price100)

Expected profit (offer all priced at $100):  53.37575757575755


This does slightly worse than offering all the products. 

One final option: let's suppose we offer the 10 products with the highest marginal profit.

In [12]:
# Use the numpy function argsort to sort the array of marginal profits:
profit_order = np.argsort(marginal_profit)

# Print the indices that order the profits:
print("profit_order: ", profit_order)

# See what happens when we index marginal_profit according to profit_order:
marginal_profit[profit_order]

# Profits are sorted in increasing order.
# Let's flip them:
profit_order = np.flip(profit_order, axis = 0)
print("profit_order: ", profit_order)

# The top 10 products in marginal profit can be obtained by using negative indices:
S_topprofit = profit_order[0:10]
print("Top 10 products in profit: ", S_topprofit)
print("Their profits: ", marginal_profit[S_topprofit])

# # Calculate expected profit:
exp_profit_topprofit, choice_probabilities_topprofit = expected_profit( S_topprofit )
print("Expected profit (offer top 10 profit products): ", exp_profit_topprofit)

print("Min utility for price: ", partworths[:,0].min())
print("Max utility for price: ", partworths[:,0].max())

print( print_attributes( grand_product_matrix[:,16]))

profit_order:  [ 5  3 12 13  7 99 80 96  9  8 10 15  2 85 11 97 98  6  4 95  1 88 82 94
 14 69 73 70 87 67 74 92 81 71  0 63 91 84 90 86 79 52 76 83 77 93 64 72
 49 68 75 51 89 45 66 62 57 78 60 58 61 65 48 25 41 54 56 33 23 59 32 27
 38 55 39 36 35 53 46 47 37 42 43 24 26 29 30 31 18 22 50 20 40 19 17 28
 44 21 34 16]
profit_order:  [16 34 21 44 28 17 19 40 20 50 22 18 31 30 29 26 24 43 42 37 47 46 53 35
 36 39 55 38 27 32 59 23 33 56 54 41 25 48 65 61 58 60 78 57 62 66 45 89
 51 75 68 49 72 64 93 77 83 76 52 79 86 90 84 91 63  0 71 81 92 74 67 87
 70 73 69 14 94 82 88  1 95  4  6 98 97 11 85  2 15 10  8  9 96 80 99  7
 13 12  3  5]
Top 10 products in profit:  [16 34 21 44 28 17 19 40 20 50]
Their profits:  [96.5 94.5 94.  92.  91.  90.5 90.  90.  89.5 89.5]
Expected profit (offer top 10 profit products):  27.26060606060607
Min utility for price:  -3.32866666666667
Max utility for price:  -0.0033333333333333
Bag: $100, Normal, Black, Velcro Flap


This does the worst out of all of the solutions we've seen so far!

To summarize: the best solution so far is to offer either all the products (or equivalently, the set of products which are most preferred by at least one customer), which achieves an expected per-customer profit of \$59.73.

Can we do better than this?

## Using integer programming to find optimal product lines

We will now see how to formulate an integer programming problem to make this decision.

In [14]:
from gurobipy import *

m = Model()

# Create the decision variables
x = m.addVars(nProducts, vtype = GRB.BINARY )
y = m.addVars(nCustomers, nProducts+1, vtype = GRB.BINARY)

# Create the constraints:
print("Creating constraints...")
for k in range(nCustomers):
    m.addConstr( sum(y[k,i] for i in range(nProducts+1)) == 1)
    for i in range(nProducts):
        m.addConstr( y[k,i] <= x[i] )
        m.addConstr( quicksum( utilities_mat[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_mat[k,i] * x[i] + utilities_mat[k,nProducts]*(1 - x[i]))
    m.addConstr( quicksum( utilities_mat[k,j] * y[k,j] for j in range(nProducts+1)) >= utilities_mat[k,nProducts] )



# Create the objective:
print("Creating objective...")
m.setObjective( quicksum(marginal_profit[i] * 1.0/nCustomers * y[k,i] for k in range(nCustomers) for i in range(nProducts)), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()


S_optimal = [i for i in range(nProducts) if x[i].x > 0.5]
print("Optimal set of products: ", S_optimal)

optimal_profit = m.objval
print("Optimal profit: ",optimal_profit)

# Verify that optimal profit is correct:
exp_profit_S_optimal, choice_probabilities_S_optimal = expected_profit( S_optimal )
print("Optimal profit (via expected_profit): ", exp_profit_S_optimal)

Creating constraints...
Creating objective...
Update completed
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [15]:
x_val = [x[i].x for i in range(nProducts)]
print(x_val)


# Verify that optimal profit is correct:
exp_profit_S_optimal, choice_probabilities_S_optimal = expected_profit( S_optimal )
print("Optimal profit (via expected_profit): ", exp_profit_S_optimal)

rel_improvement = 100*(exp_profit_S_optimal - exp_profit_mostpref)/exp_profit_mostpref
print("Relative improvement: ", rel_improvement)

for i in choice_probabilities_S_optimal.keys():
    print("Prod " + str(i) + ": " + str(choice_probabilities_S_optimal[i]) )

AttributeError: Unable to retrieve attribute 'x'

## Solving the relaxation 

Another valuable piece of information that we can obtain from this model is the optimal value of the relaxation:


In [16]:
m_relaxed = m.relax()

m_relaxed.optimize()

relaxation_bound = m_relaxed.objval

print("Relaxation bound: ", relaxation_bound)



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

The relaxation bound is \\$81.69. This tells us that no product line can obtain a per-customer expected profit greater than \\$81.69. This is valuable because earlier, when we were testing simple product lines, we could have compared their value to this bound to see how far away from optimality we were. (The best value was \\$59.73, which is quite a bit lower; this would suggest that it is worth investing some effort to try to find a better product line.)

## Constraining the model

Knowing the optimal product line is useful, but one feature of this product line that is perhaps unattractive is that it is large -- it has ...

In [ ]:
len(S_optimal)

... 39 products in it. Timbuk2 may be interested in understanding if a similar per-customer profit can be achieved with fewer products. 

(NB: the number you see here will be machine dependent. On my 2017 era Macbook this was 37.)

To see this, let's add a constraint to the model, update and solve:

In [ ]:
m.addConstr( sum(x[i] for i in range(nProducts)) <= 5) # no more than 5 products

m.update()
m.optimize()

# Retrieve the optimal solution and objective value:
S_optimal_constrained = [i for i in range(nProducts) if x[i].x > 0.5]
print("Optimal set of products (width of product line at most 5): ", S_optimal_constrained)

optimal_profit_constrained = m.objval
print("Optimal profit (width of product line at most 5): ", optimal_profit_constrained)

We thus obtain a product line of 5 products, that achieves a per-customer profit of \\$72.82 -- still significantly higher than \\$59.73 from before.

It is interesting to see what these products are:

In [ ]:
# Print the new product line
print("Optimal product line (width = 5):")
for i in S_optimal_constrained:
    print(str(i)+ " : " +print_attributes(grand_product_matrix[:,i]))
    
# Compare this to competitive products:
print("")
print("Competitive offerings:")
for i in range(competitive_products.shape[1]):
    print("Competitor " + str(i) + " : " + print_attributes(competitive_products[:,i]) )

Do you see what the optimal product line is doing? 

(It will be easier to see in the slides...)

In [ ]:
print(partworths[:,8].mean())
print(partworths[:,8].min())
print(partworths[:,8].max())

print( (partworths[:,8] >= 0).sum() )